In [2]:
import os
import xarray as xr
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
# 1. 设置文件路径

nc_file = "/home/ninglk/Chooyu/1_albedo/2Apr25/albedo_yr_without_water/gwi_albedo_without_water_2001-2020.nc"
shp_file = "/home/ninglk/Chooyu/1_albedo/2Apr25/shpfiles/Eco-TP.shp"
output_csv = "/home/ninglk/Chooyu/1_albedo/2Apr25/eco_zone_statistics.csv"

# 2. 读取数据
print("正在读取NetCDF文件...")
ds = xr.open_dataset(nc_file)
gwi = ds['albedo']  # 假设变量名为'gwi'，请根据实际情况调整

print("正在读取生态分区SHP文件...")
eco_zones = gpd.read_file(shp_file)
print(f"找到 {len(eco_zones)} 个生态分区")



正在读取NetCDF文件...
正在读取生态分区SHP文件...
找到 8 个生态分区


In [5]:
eco_zones

,FID_tibet,OBJECTID,Shape_Leng,Shape_Area,name,FID_ecoreg,ECO_NAME,ECO_ID,eco_code,geometry
0,2,1,70.036576,69.381979,Qinghai province,5,Eastern Himalayan alpine shrub and meadows,81003.0,PA1003,"MULTIPOLYGON (((92.00375 27.47615, 91.99816 27..."
1,7,1,70.036576,69.381979,Qinghai province,12,Rock and Ice,-9999.0,Rock and Ice,"MULTIPOLYGON (((88.853 27.71541, 88.85277 27.7..."
2,3,1,70.036576,69.381979,Qinghai province,28,Eastern Himalayan broadleaf forests,40401.0,IM0401,"MULTIPOLYGON (((96.38113 28.12092, 96.37437 28..."
3,1,1,70.036576,69.381979,Qinghai province,37,Central Tibetan Plateau alpine steppe,81002.0,PA1002,"MULTIPOLYGON (((87.32252 28.77399, 87.36813 28..."
4,6,1,70.036576,69.381979,Qinghai province,95,Qaidam Basin semi-desert,81324.0,PA1324,"MULTIPOLYGON (((90.87457 36.9198, 90.87466 36...."
5,4,1,70.036576,69.381979,Qinghai province,96,alpine desert steppe,81011.0,PA1011,"MULTIPOLYGON (((90.84437 38.67004, 90.7775 38...."
6,8,1,70.036576,69.381979,Qinghai province,120,Tibetan Plateau alpine meadows,81020.0,PA1020,"POLYGON ((102.72966 35.67679, 102.72971 35.676..."
7,5,1,70.036576,69.381979,Qinghai province,153,Northeastern Himalayan subalpine conifer forests,80514.0,PA0514,"MULTIPOLYGON (((91.93248 27.47032, 91.93515 27..."


In [ ]:
# 3. 准备统计结果存储
stats_list = []

# 4. 为每个生态分区计算统计量
for idx, zone in tqdm(eco_zones.iterrows(), total=len(eco_zones), desc="处理生态分区"):
    zone_name = zone['FID_tibet']  # 假设分区ID字段为ECO_ID，请根据实际情况调整
    zone_geom = zone['geometry']
    
    # 使用salem进行空间裁剪（需要安装salem库）
    try:
        import salem
        # 裁剪数据
        zone_data = gwi.salem.roi(geometry=zone_geom)
        
        # 计算统计量
        values = zone_data.values.flatten()
        values = values[~np.isnan(values)]  # 移除NaN值
        
        if len(values) > 0:
            stats = {
                'ECO_ID': zone_name,
                'Zone_Name': zone['ECO_NAME'],  # 假设有名称字段
                'Count': len(values),
                'Mean': np.mean(values),
                'Median': np.median(values),
                'Q1': np.percentile(values, 25),
                'Q3': np.percentile(values, 75),
                'P5': np.percentile(values, 5),
                'P95': np.percentile(values, 95),
                'Min': np.min(values),
                'Max': np.max(values),
                'Std': np.std(values)
            }
            stats_list.append(stats)
        else:
            print(f"警告: 生态分区 {zone_name} 无有效数据")
            
    except ImportError:
        # 如果没有salem，使用较慢的逐点判断方法
        from shapely.geometry import Point
        
        # 获取数据坐标网格
        lons, lats = np.meshgrid(gwi.lon.values, gwi.lat.values)
        values = []
        
        # 遍历每个网格点
        for i in range(gwi.lat.size):
            for j in range(gwi.lon.size):
                point = Point(lons[i,j], lats[i,j])
                if zone_geom.contains(point):
                    val = gwi.isel(lat=i, lon=j).values
                    if not np.isnan(val):
                        values.append(val)
        
        if len(values) > 0:
            values = np.array(values)
            stats = {
                'ECO_ID': zone_name,
                'Zone_Name': zone['ECO_NAME'],
                'Count': len(values),
                'Mean': np.mean(values),
                'Median': np.median(values),
                'Q1': np.percentile(values, 25),
                'Q3': np.percentile(values, 75),
                'P5': np.percentile(values, 5),
                'P95': np.percentile(values, 95),
                'Min': np.min(values),
                'Max': np.max(values),
                'Std': np.std(values)
            }
            stats_list.append(stats)
        else:
            print(f"警告: 生态分区 {zone_name} 无有效数据")

# 5. 保存统计结果
if stats_list:
    stats_df = pd.DataFrame(stats_list)
    stats_df.to_csv(output_csv, index=False)
    print(f"\n统计结果已保存到: {output_csv}")
    
    # 打印前5行结果
    print("\n统计结果预览:")
    print(stats_df.head())
    
    # 可视化各分区中位数
    plt.figure(figsize=(12, 6))
    stats_df.sort_values('Median', ascending=False).plot.bar(x='Zone_Name', y='Median', 
                                                           legend=False, color='skyblue')
    plt.title('各生态分区GWI中位数比较', fontsize=14)
    plt.ylabel('GWI中位数')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('eco_zones_median.png', dpi=300)
    plt.show()
    print("可视化结果已保存为 eco_zones_median.png")
else:
    print("错误: 未计算任何分区的统计量")

# 6. 空间可视化（可选）
try:
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    
    fig = plt.figure(figsize=(15, 10))
    ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
    
    # 绘制生态分区
    eco_zones.plot(ax=ax, edgecolor='black', facecolor='none', linewidth=1, zorder=2)
    
    # 绘制GWI数据（示例时间点）
    sample_time = gwi.time[0]
    gwi.sel(time=sample_time).plot(ax=ax, cmap='viridis', 
                                 cbar_kwargs={'label': 'GWI'}, zorder=1)
    
    # 添加地理要素
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.set_title(f'GWI空间分布与生态分区\n{sample_time.values}', fontsize=14)
    
    plt.tight_layout()
    plt.savefig('gwi_eco_zones_map.png', dpi=300)
    plt.show()
    print("空间可视化已保存为 gwi_eco_zones_map.png")
    
except ImportError:
    print("未安装cartopy，跳过空间可视化")